# Basic concepts

## Basics

When `sdupy` is imported, it automatically installs itself in the event loop of the environment (jupyter and ipython are currently supported). It there was no mainloop existing, `run_mainloop` would need to be called at the end.  

In [1]:
import sdupy

qt.qpa.xcb: X server does not support XInput 2


jupyter python kernel detected
registering jupyter gui integration...


The fundamental thing is `sdupy.Var`. It can contain any value and it can be observed for changes.

In [2]:
v = sdupy.var(4)
print(type(v), v)

<class 'sdupy.pyreactive.var.Var'> Var(4)


It has most operators overloaded, so any operaions on it will return a new `Var`-like object with the result.

In [3]:
v_plus_1 = v + 1
print(type(v_plus_1), v_plus_1)

<class 'sdupy.pyreactive.var.SyncReactiveProxy'> 5


A variable can be set to a new value:

In [4]:
v.set(3)

Depentent variables will be recomputed:

In [5]:
print(v_plus_1)

4


Operations more complicated than calling a method or operator can be done using `reactive` decorator:

In [6]:
@sdupy.reactive
def make_seq(v):
    print("proceesing")
    return [i**2 for i in range(v)]

seq = make_seq(v)
print(seq)

proceesing
[0, 1, 4]


Existing functions can be wrapped using `reactive` decorator:

In [7]:
range_seq = sdupy.reactive(sum)(seq)
print(range_seq)

5


## Advanced topics

### Synchronization with update
Normally, values are updated in the event loop. However, in the asynchronous environmnent (see [asyncio](https://docs.python.org/3/library/asyncio.html) ) you can force to wait for the update. 

In [8]:
v.set(2)
await sdupy.pyreactive.wait_for_var()
print(v_plus_1)

3


### Side effects
You should not rely on side effects of the functions marked as `reactive`. You should take into account that:
* if more than one changes of inputs happen before the output is updated, it may be recomputed only once
* the value may not be recomputed at all, if the output variable is not used